In [54]:
import numpy as np
import pandas as pd
import cloudpickle
%config IPCompleter.greedy=True

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## UPLOAD DATASETS

In [55]:
path = '../../'

In [56]:
name_original_data = 'Leads.csv'

In [57]:
full_path = path + '/02_Data/01_Original/' + name_original_data
df = pd.read_csv(full_path,index_col=0,sep =';')

In [58]:
df

,origen,fuente,no_enviar_email,no_llamar,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,ult_actividad,ambito,ocupacion,conociste_google,conociste_revista,conociste_periodico,conociste_youtube,conociste_facebook,conociste_referencias,score_actividad,score_perfil,descarga_lm
id,,,,,,,,,,,,,,,,,,,,
660737,API,Chat,No,No,0,0.0,0,0.00,Page Visited on Website,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660728,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,Business Administration,Student,No,No,No,No,No,No,14.0,20.0,Yes
660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,Media and Advertising,Unemployed,No,No,No,No,No,No,13.0,17.0,No
660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,Select,Unemployed,No,No,No,No,No,No,15.0,18.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,IT Projects Management,Unemployed,No,No,No,No,No,No,15.0,17.0,No
579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,Media and Advertising,Unemployed,No,No,No,No,No,No,14.0,19.0,Yes
579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,Business Administration,Unemployed,No,No,No,No,No,No,13.0,20.0,Yes


#### Final Variables

In [59]:
name_final_variables = path + '/05_Results/' + 'final_variables.pickle'
pd.read_pickle(name_final_variables).sort_index().index.to_list()

['ambito_Select',
 'descarga_lm_No',
 'fuente_Direct Traffic',
 'ocupacion_Unemployed',
 'ocupacion_Working Professional',
 'paginas_vistas_visita_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_total_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_OTHER',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

In [60]:
final_variables = ['ambito',
                   'descarga_lm',
                   'score_perfil',
                   'ocupacion',
                   'paginas_vistas_visita',
                   'score_actividad',
                   'tiempo_en_site_total',
                   'ult_actividad',
                   'visitas_total',                     
                  ]

#### Drop Duplicates

In [61]:
df.drop_duplicates(inplace = True)

#### by EDA

In [62]:
df = df.loc[(df.no_llamar != "OTHER" )&(df.no_enviar_email!="Yes") & (df.ult_actividad!="Email Bounced")]

In [63]:
df.shape

(6840, 20)

#### X

In [64]:
x = df[final_variables].copy()

#### Y

In [65]:
target = 'compra'

Creating  Y.

In [66]:
y = df[target].copy()

##  PIPELINE

In [67]:
def data_quality(df):
    temp = df.astype({'visitas_total': 'Int64'})             
    
    def impute_mode(variable):
        return(variable.fillna(variable.mode()[0]))
    
    var_impute_mode = ['ocupacion','ambito']
    temp[var_impute_mode] = temp[var_impute_mode].apply(impute_mode)
    
    var_impute_value = ['descarga_lm',
                         'ult_actividad']
    value = 'anknown'
    
    temp[var_impute_value] = temp[var_impute_value].fillna(value)
    var_impute_median = ['paginas_vistas_visita','score_actividad',
                           'score_perfil','tiempo_en_site_total',\
                           'visitas_total']
    def impute_median(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))
    temp[var_impute_median] = temp[var_impute_median].apply(impute_median)
    
    def group_rear_cat(variable, criteria = 0.02):
        #frec
        frec = variable.value_counts(normalize=True)
        #Identify  lower than  criteria
        temp = [cada for cada in frec.loc[frec < criteria].index.values]
        #Change to 'OTHER'
        temp2 = np.where(variable.isin(temp),'OTHER',variable)
        return(temp2)
    var_rear_cat = ['ocupacion','ambito','ult_actividad',
                     'descarga_lm']

    for variable in var_rear_cat:
        temp[variable] = group_rear_cat(temp[variable],criteria = 0.02)
    
    
    temp['visitas_total'].clip(0,50, inplace=True)
    
    return(temp)

In [68]:
data_quality(x)

,ambito,descarga_lm,score_perfil,ocupacion,paginas_vistas_visita,score_actividad,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,
660737,Select,No,15.0,Unemployed,0.0,15.0,0,Page Visited on Website,0
660728,Select,No,15.0,Unemployed,2.5,15.0,674,Email Opened,5
660727,Business Administration,Yes,20.0,Student,2.0,14.0,1532,Email Opened,2
660719,Media and Advertising,No,17.0,Unemployed,1.0,13.0,305,OTHER,1
660681,Select,No,18.0,Unemployed,1.0,15.0,1428,Converted to Lead,2
...,...,...,...,...,...,...,...,...,...
579622,Select,No,17.0,Unemployed,2.6,14.0,1409,SMS Sent,13
579615,Business Administration,No,20.0,Unemployed,2.5,14.0,210,SMS Sent,5
579546,Media and Advertising,Yes,19.0,Unemployed,2.0,14.0,238,SMS Sent,2


In [69]:
make_data_quality = FunctionTransformer(data_quality)

### Variables Transformation

In [70]:
var_ohe = ['ambito','descarga_lm','ocupacion','ult_actividad']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

var_mms = ['paginas_vistas_visita','score_actividad','score_perfil','tiempo_en_site_total','visitas_total']
mms = MinMaxScaler()

### Make Procesing pipe

#### Column transformer

In [71]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (mms, var_mms),
    remainder='drop')

#### Procesing  Pipeline

In [72]:
pipe_prepro = make_pipeline(make_data_quality, 
                            ct)

In [73]:
model = LogisticRegression(n_jobs = -1, 
                       solver='saga',
                       penalty= 'none',
                       C= 0.75
                       )

#### Pipe Final Fit

In [74]:
pipe_fit = make_pipeline(pipe_prepro,model)

#### Save pipe final Fit

In [75]:
name_fit_pipe = 'pipe_fit.pickle'

path_pipe_fit = path + '/04_Models/' + name_fit_pipe

with open(path_pipe_fit, mode='wb') as file:
   cloudpickle.dump(pipe_fit, file)

In [76]:
pipe_execution = pipe_fit.fit(x,y)

C:\Users\luise\anaconda3\envs\leadscoring\Lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


## SAVE  PIPE

In [77]:
nombre_pipe_execution = 'pipe_execution.pickle'